In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Read the file

df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df.head()

In [ ]:
## Print the columns
df.columns

In [ ]:
## about the df

df.info()

In [ ]:
## Rename Columns 

df= df.rename(columns = {'3SsnPorch':'threeSsnPorch',
                         '1stFlrSF':'firstFlrSF',
                         '2ndFlrSF':'secondFlrSF'                     
                        }, inplace = False)

In [ ]:
## create a sql engine

# import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df.to_sql("houseprice", con =engine)



In [ ]:
## Test the query
sql='''

Select * from houseprice
limit 5


''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

# We will be breaking down this df into smaller chunks, and utilize the tables further for EDA analysis

# The chunks will be broken down into 6 tables, which individually represent each key KPI associated with it

## Table 1:-sales (KPIs related to age and value of the house)
## Table 2:-area (geographic inputs related to the house prices)
## Table 3:-exteriorbuilding (external facilities)
## Table 4:-interiorbuilding (internal facilities)
## Table 5:-garage  (garage data)
## Table 6:-miscellaneous (others)

In [ ]:
## Test the query
sql='''

Select * from houseprice
limit 5


''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

In [ ]:
##price group 
df[['SalePrice','SaleType','SaleCondition','MoSold','YrSold']].head()

In [ ]:

##area group 

df[['MSSubClass','MSZoning','LotFrontage','LotArea','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','PavedDrive']].head()

In [ ]:
##exteriorbuilding group 

df[['BldgType','HouseStyle','OverallQual','OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond','WoodDeckSF','OpenPorchSF','EnclosedPorch','threeSsnPorch','ScreenPorch','PoolArea','PoolQC','Fence']].head()

In [ ]:
##interiorbuilding group 

df[['Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','Heating','HeatingQC','CentralAir','Electrical','firstFlrSF','secondFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','Fireplaces','FireplaceQu']].head()

In [ ]:
## garage group
df[['GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageArea','GarageQual','GarageCond']].head()

In [ ]:
## miscellaneous group

df[['MiscFeature','MiscVal']].head()

In [ ]:
## Creating the tables using with clause (for demo we are creating only 2 tables, if you'd like to me update the NB with the full query, please feel free to comment in the NB)


sql='''

with sales as 
(
select
 row_number () over (order by (select null)) as row,
 SalePrice,
 MSSubClass,
 MSZoning,
 LotFrontage,
 LotArea,
 Street,
 Alley,
 LotShape,
 LandContour,
 Utilities,
 LotConfig,
 LandSlope,
 Neighborhood,
 Condition1,
 Condition2
 from 
 houseprice
),

exteriorbuilding as
(
select
row_number () over ( order by BldgType,
HouseStyle,
OverallQual,
OverallCond,
YearBuilt,
YearRemodAdd,
RoofStyle,
RoofMatl,
Exterior1st,
Exterior2nd,
MasVnrType,
MasVnrArea,
ExterQual,
ExterCond,
WoodDeckSF,
OpenPorchSF,
EnclosedPorch,
threeSsnPorch,
ScreenPorch,
PoolArea,
PoolQC,
Fence asc) as row,
BldgType,
HouseStyle,
OverallQual,
OverallCond,
YearBuilt,
YearRemodAdd,
RoofStyle,
RoofMatl,
Exterior1st,
Exterior2nd,
MasVnrType,
MasVnrArea,
ExterQual,
ExterCond,
WoodDeckSF,
OpenPorchSF,
EnclosedPorch,
threeSsnPorch,
ScreenPorch,
PoolArea,
PoolQC,
Fence
from
houseprice)

select * from sales 
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


## This is one of the best practices, we can adhere to while handling datasets with large column sizes, for the benfit of the demo we have created only 2 tables, please feel free to comment in the NB if you'd like the full  query